In [3]:
import requests
import os
import pandas as pd
import numpy as np

In [4]:
host = 'http://143.248.23.236:4042/'
route = "maxentscan"
data ={"seq": "NM_000368.5(TSC1):c.364-1G>T", "session_data": "kdw59520@kaist.ac.kr"}
response = requests.post(
    url=f'{host}{route}',
    json=data,
    headers={"Content-Type": "application/json; charset=UTF-8"}
)
response_data = response.json()


In [5]:
maxentscan_with_strand = {}
for strand in response_data["resources"].keys():
    maxentscan_with_strand[strand] = { key: value for item in response_data["resources"][strand] for key, value in [tuple(item.values())]}

    

In [4]:
host = 'http://143.248.208.160:4042/'
route = "parse2mane"
seq="chr1:925952-925952 G>A"
#seq="chrX:18609570-18609570 G>A"
#seq="NM_000368.5(TSC1):c.364-1G>T"
data ={"seq": seq, "session_data": "kdw59520@kaist.ac.kr"}
response = requests.post(
    url=f'{host}{route}',
    json=data,
    headers={"Content-Type": "application/json; charset=UTF-8"}
)
response_data = response.json()

In [5]:
response_data

{'resources': [{'Gene': 'SAMD11<br><a href=https://www.genecards.org/cgi-bin/carddisp.pl?gene=SAMD11 target="_blank">GeneCards</a><br><a href=https://gtexportal.org/home/gene/SAMD11 target="_blank">GTEx</a>',
   'HGVS': '<a href=https://varsome.com/variant/hg38/ENST00000616016.5%3Ac.548G%3EA?annotation-mode=germline target="_blank">(ENST00000616016.5):c.548G>A</a><br>(p.Gly183Glu)<br><br><a href=https://varsome.com/variant/hg38/NM_001385641.1%3Ac.548G%3EA?annotation-mode=germline target="_blank">(NM_001385641.1):c.548G>A</a><br>(p.Gly183Glu)',
   'Strand': '+ (plus strand)',
   'Transcript ID': 'ENST00000616016.5<br><br>NM_001385641.1(refseq)',
   'Variant': 'chr1:925952-925952 G>A<br><a href=http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg38&lastVirtModeType=default&lastVirtModeExtraState=&virtModeType=default&virtMode=0&nonVirtPosition=&position=chr1%3A925952%2D925952&hgsid=1156735329_9yjwvrj3bzbN4iAJuhQElyCAtvvR target ="_blank">UCSC genome browser</a><br><a href=https://ensembl.org/Ho

In [7]:
import json
import pprint
consequence_data_path ="../ensembl_variant_consequences.json"
with open(consequence_data_path) as f:
    consequence_datas=json.load(f)
consequence_datas[0]

{'so_term': 'transcript_ablation',
 'so_description': 'A feature ablation whereby the deleted region includes a transcript feature',
 'so_accession': 'SO:0001893',
 'display_term': 'Transcript ablation',
 'impact': 'HIGH',
 'color': '#ff0000'}

In [8]:
for item in response_data["resources"]:
    variant_types = item["Variant Type"].split("&")
    variant_types_array = []
    for variant_type in variant_types:
        for consequence_data in consequence_datas:
            if variant_type == consequence_data["so_term"]:
                button_html = f"""
                            <button class="button" style="background-color: {consequence_data['color']};" onclick="this.focus();">
                                { consequence_data['display_term'] }
                                <div class="tooltip-consequence">
                                    <strong>Impact:</strong> {consequence_data['impact'] }<br>
                                    <strong>Description:</strong> { consequence_data['so_description'] }
                                </div>
                            </button>
                            """
                #print(button_html)
                variant_types_array.append(button_html)
    join_button_html = "<br>".join(variant_types_array)
    print(join_button_html)

In [9]:
host = 'http://143.248.23.236:4042/'
route = "split2vcf"
seq="chrX:18609570-18609570 G>A"
#seq="NM_000368.5(TSC1):c.364-1G>T"
data ={"seq": seq, "session_data": "kdw59520@kaist.ac.kr"}
response = requests.post(
    url=f'{host}{route}',
    json=data,
    headers={"Content-Type": "application/json; charset=UTF-8"}
)
response_data = response.json()

In [10]:
response_data['ensembl']

[{'APPRIS': '-',
  'Canonical': '-',
  'Clinical Significance': 'pathogenic&likely_pathogenic',
  'Existing Variation': '<a href=https://www.ncbi.nlm.nih.gov/snp/rs267608653&CM085312?horizontal_tab=true target="_blank">rs267608653&CM085312</a>',
  'Exon': '15/22',
  'Gene': 'CDKL5',
  'Gene ID': 'ENSG00000008086',
  'HGVSc': 'ENST00000379989.6:c.2152G>A',
  'HGVSp': 'ENSP00000369325.3:p.Val718Met',
  'Intron': '-',
  'MANE Select': '-',
  'Mane Plus Clinical': '-',
  'PolyPhen2': 'probably_damaging(0.994)',
  'REVEL': 'Pathogenic(0.625)',
  'SIFT': 'deleterious_low_confidence(0)',
  'SIFT4G': '-',
  'Strand': '+',
  'Transcript ID': 'ENST00000379989.6',
  'Variant Type': '\n                                <button class="button-consequence" style="background-color: #ffd700;" onclick="this.focus();">\n                                    Missense variant\n                                    <div class="tooltip-consequence">\n                                        <strong>Impact:</strong>

In [13]:
import requests
import pprint
def search_litvar(variant_id):
    """
    NCBI LitVar API를 사용하여 특정 variant_id에 대한 정보를 검색합니다.

    Parameters:
        variant_id (str): 검색하고자 하는 변이의 ID (예: 'rs121913527')

    Returns:
        dict: API로부터 받은 JSON 응답을 파이썬 딕셔너리로 반환
    """
    base_url = 'https://www.ncbi.nlm.nih.gov/research/litvar2-api/search/'
    params = {
        'variant': f'litvar@{variant_id}##',
        'filters': '{}',
        'sort': 'score desc',
        'page': 2,
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # HTTP 오류가 발생하면 예외를 발생시킴
        return response.json()
    except requests.exceptions.HTTPError as errh:
        print("HTTP 오류:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("연결 오류:", errc)
    except requests.exceptions.Timeout as errt:
        print("시간 초과 오류:", errt)
    except requests.exceptions.RequestException as err:
        print("요청 오류:", err)

# 사용 예시
if __name__ == "__main__":
    variant_id = 'rs121913527'
    result = search_litvar(variant_id)
    pprint.pprint(result)

{'accessions': ['litvar@rs121913527##'],
 'count': 775,
 'current': 2,
 'facets': {'count': 775,
            'journal': {'buckets': [{'count': 59, 'val': 'Oncotarget'},
                                    {'count': 46, 'val': 'Cancers (Basel)'},
                                    {'count': 46, 'val': 'PLoS One'},
                                    {'count': 34, 'val': 'Nat Commun'},
                                    {'count': 24, 'val': 'Front Oncol'}]},
            'section': {'count': 870,
                        'snippets': {'buckets': [{'count': 276,
                                                  'cp': 276,
                                                  'val': 'table'},
                                                 {'count': 214,
                                                  'cp': 214,
                                                  'val': 'results'},
                                                 {'count': 99,
                                                  

In [1]:
import requests

def search_litvar_all(variant_id):
    """
    NCBI LitVar API를 사용하여 특정 variant_id에 대한 모든 결과를 가져옵니다.

    Parameters:
        variant_id (str): 검색하고자 하는 변이의 ID (예: 'rs121913527')

    Returns:
        list: 모든 결과를 포함하는 리스트
    """
    page = 1
    base_url = 'https://www.ncbi.nlm.nih.gov/research/litvar2-api/search/'
    params = {
        'variant': f'litvar@{variant_id}##',
        'filters': '{}',
        'sort': 'score desc',
        'page': page,  # 페이지당 최대 결과 수 (API가 허용하는 최대값)
        'page_size': 20
    }

    all_results = []
    offset = 1  # 결과를 가져올 시작 위치

    try:
        while True:
            params['page'] = offset
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            total_pages = data['total_pages']
            facets = data['facets']
            

            # 첫 번째 요청에서 총 결과 수를 가져옴
            print(total_pages, offset, len(data['results']))
            all_results.append(data['results'])
            if total_pages == offset:
                pprint.pprint(facets)
                break
            offset+=1
        return all_results, facets

    except requests.exceptions.HTTPError as errh:
        print("HTTP 오류:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("연결 오류:", errc)
    except requests.exceptions.Timeout as errt:
        print("시간 초과 오류:", errt)
    except requests.exceptions.RequestException as err:
        print("요청 오류:", err)

# 사용 예시
if __name__ == "__main__":
    variant_id = 'rs121913527'
    results, facets = search_litvar_all(variant_id)
    print(f"총 {len(results)}개의 결과를 가져왔습니다.")
    # 결과를 출력하거나 파일로 저장할 수 있습니다.

78 1 10
78 2 10
78 3 10
78 4 10
78 5 10
78 6 10
78 7 10
78 8 10
78 9 10
78 10 10
78 11 10
78 12 10
78 13 10
78 14 10
78 15 10
78 16 10
78 17 10
78 18 10
78 19 10
78 20 10
78 21 10
78 22 10
78 23 10
78 24 10
78 25 10
78 26 10
78 27 10
78 28 10
78 29 10
78 30 10
78 31 10
78 32 10
78 33 10
78 34 10
78 35 10
78 36 10
78 37 10
78 38 10
78 39 10
78 40 10
78 41 10
78 42 10
78 43 10
78 44 10
78 45 10
78 46 10
78 47 10
78 48 10
78 49 10
78 50 10
78 51 10
78 52 10
78 53 10
78 54 10
78 55 10
78 56 10
78 57 10
78 58 10
78 59 10
78 60 10
78 61 10
78 62 10
78 63 10
78 64 10
78 65 10
78 66 10
78 67 10
78 68 10
78 69 10
78 70 10
78 71 10
78 72 10
78 73 10
78 74 10
78 75 10
78 76 10
78 77 10
78 78 6


NameError: name 'pprint' is not defined

In [66]:
new_results = sum(results, [])

In [72]:
test=0
for item in new_results:
    if item['year']==2018:
        test+=1
test

77

In [80]:
def get_years(results):
    return list(set([result['year'] for result in results]))

def sort_dict_by_years(results):
    years = get_years(results)
    set_dict_by_year = {}
    for year in years:
        result_by_year = [] 
        for result in results:
            if result['year'] == year:
                result_by_year.append(result)
        set_dict_by_year[year] = result_by_year
    return set_dict_by_year 
                
    
test_dict=sort_dict_by_years(results=new_results)
    

In [85]:
len(test_dict[2018])

77

In [1]:
import requests
import pprint
import concurrent.futures

def fetch_page(base_url, params, page):
    params['page'] = page
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.json()

def search_litvar_all(variant_id):
    """
    NCBI LitVar API를 사용하여 특정 variant_id에 대한 모든 결과를 가져옵니다.

    Parameters:
        variant_id (str): 검색하고자 하는 변이의 ID (예: 'rs121913527')

    Returns:
        list: 모든 결과를 포함하는 리스트
    """
    base_url = 'https://www.ncbi.nlm.nih.gov/research/litvar2-api/search/'
    params = {
        'variant': f'litvar@{variant_id}##',
        'filters': '{}',
        'sort': 'score desc',
        'page': 1,  # 페이지당 최대 결과 수 (API가 허용하는 최대값)
        'page_size': 20
    }

    try:
        # 첫 번째 요청으로 total_pages를 가져옴
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        total_pages = data['total_pages']
        facets = data['facets']

        print(f"총 페이지 수: {total_pages}")

        all_results = []

        # ThreadPoolExecutor를 사용하여 여러 페이지를 동시에 요청
        with concurrent.futures.ThreadPoolExecutor(max_workers=total_pages) as executor:
            future_to_page = {executor.submit(fetch_page, base_url, params, page): page for page in range(1, total_pages + 1)}
            for future in concurrent.futures.as_completed(future_to_page):
                page = future_to_page[future]
                try:
                    data = future.result()
                    all_results.extend(data['results'])
                except Exception as exc:
                    print(f"{page} 페이지에서 예외 발생: {exc}")

        pprint.pprint(facets)
        return all_results, facets

    except requests.exceptions.HTTPError as errh:
        print("HTTP 오류:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("연결 오류:", errc)
    except requests.exceptions.Timeout as errt:
        print("시간 초과 오류:", errt)
    except requests.exceptions.RequestException as err:
        print("요청 오류:", err)

# 사용 예시
if __name__ == "__main__":
    variant_id = 'rs121913527'
    results, facets = search_litvar_all(variant_id)
    print(f"총 {len(results)}개의 결과를 가져왔습니다.")
    # 결과를 출력하거나 파일로 저장할 수 있습니다.

총 페이지 수: 78
{'count': 777,
 'journal': {'buckets': [{'count': 59, 'val': 'Oncotarget'},
                         {'count': 46, 'val': 'Cancers (Basel)'},
                         {'count': 46, 'val': 'PLoS One'},
                         {'count': 34, 'val': 'Nat Commun'},
                         {'count': 24, 'val': 'Front Oncol'}]},
 'section': {'count': 874,
             'snippets': {'buckets': [{'count': 276, 'cp': 276, 'val': 'table'},
                                      {'count': 215,
                                       'cp': 215,
                                       'val': 'results'},
                                      {'count': 100,
                                       'cp': 100,
                                       'val': 'discussion'},
                                      {'count': 82, 'cp': 82, 'val': 'other'},
                                      {'count': 66, 'cp': 66, 'val': 'methods'},
                                      {'count': 50,
                 

In [11]:
facets

{'count': 776,
 'journal': {'buckets': [{'val': 'Oncotarget', 'count': 59},
   {'val': 'Cancers (Basel)', 'count': 46},
   {'val': 'PLoS One', 'count': 46},
   {'val': 'Nat Commun', 'count': 34},
   {'val': 'Front Oncol', 'count': 24}]},
 'year': {'buckets': [{'val': 2022, 'count': 97},
   {'val': 2019, 'count': 85},
   {'val': 2020, 'count': 83},
   {'val': 2018, 'count': 77},
   {'val': 2021, 'count': 76},
   {'val': 2023, 'count': 68},
   {'val': 2017, 'count': 67},
   {'val': 2016, 'count': 57},
   {'val': 2015, 'count': 55},
   {'val': 2024, 'count': 45},
   {'val': 2014, 'count': 30},
   {'val': 2013, 'count': 16},
   {'val': 2012, 'count': 7},
   {'val': 2011, 'count': 5},
   {'val': 2010, 'count': 4},
   {'val': 2009, 'count': 3},
   {'val': 2006, 'count': 1}]},
 'section': {'count': 871,
  'snippets': {'buckets': [{'val': 'table', 'count': 276, 'cp': 276},
    {'val': 'results', 'count': 214, 'cp': 214},
    {'val': 'discussion', 'count': 99, 'cp': 99},
    {'val': 'other', 'c

In [ ]:
import requests
import pprint
import concurrent.futures

def fetch_page(base_url, params, page):
    params['page'] = page
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.json()

def search_litvar_all(variant_id):
    """
    NCBI LitVar API를 사용하여 특정 variant_id에 대한 모든 결과를 가져옵니다.

    Parameters:
        variant_id (str): 검색하고자 하는 변이의 ID (예: 'rs121913527')

    Returns:
        list: 모든 결과를 포함하는 리스트
    """
    base_url = 'https://www.ncbi.nlm.nih.gov/research/litvar2-api/search/'
    params = {
        'variant': f'litvar@{variant_id}##',
        'filters': '{}',
        'sort': 'score desc',
        'page': 1,  # 페이지당 최대 결과 수 (API가 허용하는 최대값)
        'page_size': 20
    }

    try:
        # 첫 번째 요청으로 total_pages를 가져옴
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        total_pages = data['total_pages']
        facets = data['facets']

        print(f"총 페이지 수: {total_pages}")

        all_results = []

        # ThreadPoolExecutor를 사용하여 여러 페이지를 동시에 요청
        with concurrent.futures.ThreadPoolExecutor(max_workers=total_pages) as executor:
            future_to_page = {executor.submit(fetch_page, base_url, params, page): page for page in range(1, total_pages + 1)}
            for future in concurrent.futures.as_completed(future_to_page):
                page = future_to_page[future]
                try:
                    data = future.result()
                    all_results.extend(data['results'])
                except Exception as exc:
                    print(f"{page} 페이지에서 예외 발생: {exc}")

        #pprint.pprint(facets)
        print(type(all_results))
        if all_results == None:
            raise ValueError("No data in all_results")
        if all_results == []:
            raise ValueError("No data in all_results")
        return all_results, facets

    except requests.exceptions.HTTPError as errh:
        print("HTTP 오류:", errh)
        return [], []
    except requests.exceptions.ConnectionError as errc:
        print("연결 오류:", errc)
        return [], []
    except requests.exceptions.Timeout as errt:
        print("시간 초과 오류:", errt)
        return [], []
    except requests.exceptions.RequestException as err:
        print("요청 오류:", err)
        return [], []
    except ValueError as errv:
        print("ValueError:", errv)
        return [], []


def response2rsid(response_data):
    try:
        each_rsid = ''
        for response_key in response_data.keys():
            each_resource_data = response_data[response_key]
            each_data = each_resource_data[0]
            if len(each_resource_data) == 0:
                raise Exception("No data in response_data")
            ex_variants = each_data['ex_variants'].split("&")
            #print(ex_variants)
            if ((ex_variants[0] == '-') or (ex_variants[0] == '')):
                raise Exception('do not support this type of ex_variants')
            each_rsid = ''
            for item in ex_variants:
                if item.startswith('rs'):
                    each_rsid = item
                    break
            #print(each_rsid)
        return each_rsid
    except Exception as e:
        print(e)
    
def process_litvar(variant, 
                   host='http://143.248.208.160:4042/', 
                   route="parse2mane", 
                   session_data="kdw59520@kaist.ac.kr",
                   headers={"Content-Type": "application/json; charset=UTF-8"}):

    data = {"seq": variant, "session_data": session_data}
    response = requests.post(
        url=f'{host}{route}',
        json=data,
        headers=headers)
    response_data = response.json()
    rsid = response2rsid(response_data=response_data)
    results, facets = search_litvar_all(variant_id=rsid)
    
    print(f"총 {len(results)}개의 결과를 가져왔습니다.")
    return results, facets

# 결과를 출력하거나 파일로 저장할 수 있습니다.
#seq="chr1:925952-925952 G>A"
#seq="chrX:18609570-18609570 G>A"
#seq="NM_000368.5(TSC1):c.364-1G>T"
#seq="chr3:155116987-155116987 G>A" #support to cosv id
seq="chr12:25225628-25225628 C>G"
results, facets = process_litvar(variant=seq)
#pprint.pprint(results)

HTTP 오류: 400 Client Error: Bad Request for url: https://www.ncbi.nlm.nih.gov/research/litvar2-api/search/?variant=litvar%40rs1640863258%23%23&filters=%7B%7D&sort=score+desc&page=1&page_size=20
총 0개의 결과를 가져왔습니다.


In [4]:
import json
with open('./test.json', 'w') as f:
    json.dump(results, f)

In [3]:
facets

{'count': 10,
 'journal': {'buckets': [{'val': 'Genet Med', 'count': 2},
   {'val': 'Dev Med Child Neurol', 'count': 1},
   {'val': 'EMBO J', 'count': 1},
   {'val': 'J Med Genet', 'count': 1},
   {'val': 'Neural Plast', 'count': 1}]},
 'year': {'buckets': [{'val': 2017, 'count': 2},
   {'val': 2018, 'count': 2},
   {'val': 2021, 'count': 2},
   {'val': 2012, 'count': 1},
   {'val': 2015, 'count': 1},
   {'val': 2016, 'count': 1},
   {'val': 2019, 'count': 1}]},
 'section': {'count': 9,
  'snippets': {'buckets': [{'val': 'table', 'count': 4, 'cp': 4},
    {'val': 'figure_legend', 'count': 2, 'cp': 2},
    {'val': 'results', 'count': 2, 'cp': 2},
    {'val': 'introduction', 'count': 1, 'cp': 1}]}}}

In [1]:
from variants import Variants

In [2]:
Variants("rs121913527").variant2locus()

['NC_000012.12', 'g.25225628C>A']


('chr12:25225628-25225628', 'C', 'A')